In [1]:
import numpy as np
import os
import copy

import open3d as o3d
from open3d import JVisualizer

In [16]:
class Mesh:
    def __init__(self, filepath):
        os.path.isfile(filepath)
        self.mesh_o3d = o3d.io.read_triangle_mesh(filepath)
        self.print_mesh()

    def visualize(self):
        self.mesh_o3d.compute_vertex_normals()
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.get_render_option().mesh_show_back_face = True
        self.add_to_vis(vis)
        mesh_frame = o3d.geometry.create_mesh_coordinate_frame(size=4,
                                                               origin=[0, 0, 0])
        vis.add_geometry(mesh_frame)
        vis.run()
        vis.destroy_window()

    def add_to_vis(self, vis):
        """ Add the mesh to the visualization `vis` object """
        self.mesh_o3d.compute_vertex_normals()
        vis.add_geometry(self.mesh_o3d)

    def print_mesh(self):
        print("Testing mesh in open3d ...")
        print(self.mesh_o3d)
        print(np.asarray(self.mesh_o3d.vertices))
        print(np.asarray(self.mesh_o3d.triangles))
        print("")
    
    def transform(self, T):
        self.mesh_o3d.transform(T)

In [17]:
def quaternion_to_matrix(q):
    # q: numpy array: [x, y, z, w]
    x = q[0]
    y = q[1]
    z = q[2]
    w = q[3]
    Nq = w*w + x*x + y*y + z*z
    if Nq < np.finfo(np.float).eps:
        return np.eye(3)
    s = 2.0/Nq
    X = x*s
    Y = y*s
    Z = z*s
    wX = w*X; wY = w*Y; wZ = w*Z
    xX = x*X; xY = x*Y; xZ = x*Z
    yY = y*Y; yZ = y*Z; zZ = z*Z
    return np.array(
           [[ 1.0-(yY+zZ), xY-wZ, xZ+wY ],
            [ xY+wZ, 1.0-(xX+zZ), yZ-wX ],
            [ xZ-wY, yZ+wX, 1.0-(xX+yY) ]])

In [18]:
# FILL PATHS BELOW
gt_mesh_path = "/home/yunchang/Downloads/warty_tiny_loop5_gt.ply"
est_mesh_path = "/home/yunchang/Downloads/tesse_yunchang_lambda_23073_1000743885426201409.ply"

In [19]:
# FILL IN 
robot_0_position_0 = np.array([91.996468, -49.997837, -0.271650])
robot_0_quaternion_0 = np.array([0, 0, 0.945753, 0.324743])

In [20]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 2881891 points and 3109646 triangles.
[[ 9.92750e+01 -3.51299e+01  7.50000e-02]
 [ 9.92666e+01 -3.51250e+01  7.50000e-02]
 [ 9.92750e+01 -3.51283e+01  2.50000e-02]
 ...
 [ 6.22250e+01 -4.64093e+01  1.52500e+00]
 [ 6.20750e+01 -4.66750e+01  1.58768e+00]
 [ 6.20750e+01 -4.66750e+01  1.58768e+00]]
[[      0       1       2]
 [      2       1       3]
 [      4       5       0]
 ...
 [2881825  148006  148025]
 [ 148025 2881846 2881825]
 [ 148045 2881846  148025]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 3591187 points and 3737015 triangles.
[[18.425     26.4218    -0.575    ]
 [18.422     26.425     -0.575    ]
 [18.425     26.425     -0.588612 ]
 ...
 [ 8.675     23.225     -0.0967604]
 [ 8.725     23.225     -0.0958508]
 [ 8.775     23.225     -0.0955864]]
[[      0       1       2]
 [      3       1       0]
 [      4       5       6]
 ...
 [3591184 3591103 359118

In [21]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [22]:
# Transform gt mesh to est mesh 
w_T_gtPose0 = np.eye(4)
w_T_gtPose0[:3, :3] = quaternion_to_matrix(robot_0_quaternion_0)
w_T_gtPose0[:3, 3] = robot_0_position_0

w_T_estPose0 = np.eye(4) # first pose of first robot after dpgo is identity 

est_T_gt = np.dot(np.linalg.inv(w_T_gtPose0), w_T_estPose0)
gt_mesh.transform(est_T_gt)

In [23]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [24]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [25]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [26]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [27]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [28]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [29]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [30]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 0.975996, inlier_rmse = 0.706754, and correspondence_set size of 3504985
Access transformation to get result.


In [31]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [32]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 0.970207, inlier_rmse = 0.318010, and correspondence_set size of 3484193
Access transformation to get result.

Transformation is:
[[ 0.99963777 -0.00956201 -0.02515728  0.63998349]
 [ 0.00929708  0.99990031 -0.01062698 -0.66334041]
 [ 0.02525638  0.01038924  0.99962702  0.07886306]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 3484193 elements.
Use numpy.asarray() to access data.



In [33]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [34]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [35]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [36]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [37]:
calc_corresp(est_pcl, gt_pcl, correspondences)

Positive color matches:  3179963
Negative color matches:  304230
Total correspondences:  3484193
Positive: 91.26827934043837  % 
Negative: 8.731720659561626  % 
